In [1]:
import math
import numpy as np
import pandas as pd
import ortools
import itertools

from typing import Generator, List, Tuple

In [2]:
# look at all possible cut assignments that result in the needed quantities
#   minimize over all of the cut assignments
from ortools.sat.python import cp_model
from ortools.linear_solver import pywraplp


In [69]:
fn = "./#5 Consolidated.csv"

In [70]:
df = pd.read_csv(fn, header=None)
df.columns = ["length", "quantity"]

base_length = 240
waste_tolerance = 0
desired_efficiency_stopping_point = None  #.95 # one can set this to instruct the solver to stop after a certain efficiency is reached

exact_cuts = df[df["length"] == base_length].copy()

# df = df[df["length"] < base_length]
needed_lengths = df["length"].values
needed_quantities = df["quantity"].values

# We expect to be allowed to make cuts
# 
# allowed_cuts = [
#   (3,), (3, 3,), (3, 3, 3,), (3, 5,), (5,), (5, 3,), (5, 5,), (8,),
# ]

In [71]:
len(df)

9

In [72]:
df["quantity"].sum()

146

In [73]:
df.sort_values("length", ascending=False)

,length,quantity
6,184,4
7,148,4
5,127,30
4,108,26
8,103,4
3,96,8
2,69,26
1,66,18
0,45,26


In [74]:
lengths_to_quantities = {
    r["length"]: r["quantity"] for _, r in df.iterrows()
}
total_length_to_produce = sum(
    l * q for l, q in zip(needed_lengths, needed_quantities)
)

The naive method of candidate generation that we have used before seems to scale very poorly with the number of inputs (exponentially, as we iterate over all combinations).

We should probably use a heuristic to build them. But first we will need to re-frame the problem a bit.

Let $c_1 = (x_1, \ldots, x_p), c_2 = (y_1, \ldots, y_q)$ be two cuts with $c_1 \neq c_2$. We can define the _multiplicity function_ $N(c, l)$ which counts the number of times length $l$ appears in cut $c$. We shall say that $c_2$ _dominates_ $c_1$ if for all lengths $l$, $N(c_2, l) \geq N(c_1, l)$. This effectively means that $c_2$ contains all of the segments that would be produced by $c_1$. 

If we frame our problem so that we only consider cuts which are dominated by no other cuts, then we will be able to run our solver and modify the way that we measure waste. The hope is that we are able to greatly reduce the number of candidates that we generate in this manner.

In [75]:
def _is_sorted(x):
    return sorted(x) == x

def _generate_non_dominated_cuts(
    cut: List[float], 
    needed_lengths: List[float], 
    length_remaining: float, 
    tolerance_per_cut: float,
):  
    
    if not _is_sorted(cut):
        return
    
    if length_remaining < min(needed_lengths):
        yield cut
    
    for length in needed_lengths:
        excess = length_remaining - length - tolerance_per_cut
        if excess >= 0 or not cut:
            yield from _generate_non_dominated_cuts(
                cut + [length], 
                needed_lengths, 
                excess, 
                tolerance_per_cut=tolerance_per_cut,
            )
            
def get_possible_cuts(needed_lengths: List[float], base_length: float, tolerance_per_cut: float = 0):
    assert max(needed_lengths) <= base_length
    
    return (
        tuple(x)
            for x in _generate_non_dominated_cuts(
                [], 
                needed_lengths, 
                length_remaining=base_length,
                tolerance_per_cut=tolerance_per_cut,
            )
    )

In [91]:
possible_cuts = list(get_possible_cuts(needed_lengths, base_length, tolerance_per_cut=0.125))

In [92]:
len(possible_cuts)

40

In [93]:
cut_produced_quantities = {}
for i, cut in enumerate(possible_cuts):
    for j, length in enumerate(needed_lengths):
        cut_produced_quantities[i, j] = cut.count(length)

Here, we define our optimality slightly differently.

We permit the solver to find solutions in which we overproduce (as now that we consider only dominant cuts, we can't restrict to equality, as it may not be feasible).

Thus, the new way we're itemizing waste is to sum:

* All excess length produced by unneeded cuts
* All waste unavoidable to the cuts (as in cut wastes).

We will also come up with a smarter bound for how many times a cut is needed. For every cut, we know the multiplicity of lengths produced from the multiplicity function. Let $q(l)$ be the quantity of pieces of length $l$ needed. We can always bound by

$$
max_{l} \left\lbrace \left\lceil\frac{q(l)}{N(c, l)}\right\rceil : N(c, l) > 0 \right\rbrace
$$

In [94]:
model = pywraplp.Solver.CreateSolver('SCIP') # cp_model.CpModel()
naive_max_cuts = max(needed_quantities)

cut_vars = {}
for i, cut in enumerate(possible_cuts):
    max_needed = max(
        math.ceil(lengths_to_quantities[length] / cut_produced_quantities[i, j])
            for j, length in enumerate(needed_lengths) if cut_produced_quantities[i, j] > 0
    )
    cut_vars[i] = model.IntVar(0, max_needed, f"Cut according to rule {cut}")

cut_wastes = {}
for i, cut in enumerate(possible_cuts):
    cut_wastes[i] = base_length - sum(cut)

segments_produced = {}
for (j, length), needed in zip(enumerate(needed_lengths), needed_quantities): 
    for i, cut in enumerate(possible_cuts):
        segments_produced[i, j] = cut_vars[i] * cut_produced_quantities[i, j]  

# assert that we produce at least as much as we need
for (j, length), needed in zip(enumerate(needed_lengths), needed_quantities):
    total_produced = sum(
        segments_produced[i, j] for i, _ in enumerate(possible_cuts)
    )
    model.Add(total_produced >= needed)

# measure the amount of waste that we produced
total_excess = []
for (j, length), needed in zip(enumerate(needed_lengths), needed_quantities):
    produced = sum(
        segments_produced[i, j] for i, _ in enumerate(possible_cuts)
    )
    total_excess.append((produced - needed) * length)
total_excess = sum(total_excess)

unavoidable_waste = sum(
    var * cut_wastes[i] for i, var in cut_vars.items()
)

total_waste = unavoidable_waste + total_excess

if desired_efficiency_stopping_point is not None:
    print(f"setting solver to stop if efficiency of {desired_efficiency_stopping_point*100:.2f}% is reached.")    
    model.Add(
        total_waste
            <= int((1 - desired_efficiency_stopping_point) * total_length_to_produce)
    )
else:
    print(f"minimizing total waste produced by cutting process")
    model.Minimize(total_waste)

minimizing total waste produced by cutting process


In [95]:
status = model.Solve()
if status == model.OPTIMAL:
    print("solver found optimal solution")
else: 
    raise ValueError("no guarantee of solution optimality")

solver found optimal solution


In [96]:
solution_df = []
for i, var in cut_vars.items():
    solution_df.append(
        {
            "cut": possible_cuts[i],
            "cut_idx": i,
            # "number": solver.Value(var),
            "number": var.solution_value(),
        }
    )

solution_df = pd.DataFrame(solution_df)
solution_df["waste"] = solution_df["cut_idx"].apply(cut_wastes.get)
solution_df["total_waste"] = solution_df["waste"] * solution_df["number"]

In [97]:
confirmed_counts = {}
# confirm that we didn't fuck it
for i, var in cut_vars.items():
    for j, length in enumerate(needed_lengths):
        # confirmed_counts[length] = confirmed_counts.get(length, 0) + cut_produced_quantities[i, j] * solver.Value(var)
        confirmed_counts[length] = confirmed_counts.get(length, 0) + cut_produced_quantities[i, j] * var.solution_value()
        
confirmed_counts_df = pd.DataFrame(
    [{"length": length, "quantity": quantity} for length, quantity in confirmed_counts.items()]
)

In [98]:
confirmatory_df = df.merge(
    confirmed_counts_df,
    how="left",
    on="length",
    suffixes=["", "_produced_by_algorithm"],
)

assert (confirmatory_df["quantity"] <= confirmatory_df["quantity_produced_by_algorithm"]).all()

In [99]:
confirmatory_df["excess_quantity_produced"] = (
    confirmatory_df["quantity_produced_by_algorithm"] - confirmatory_df["quantity"]
)
confirmatory_df["excess_length_produced"] = (
    confirmatory_df["excess_quantity_produced"] * confirmatory_df["length"]
)

In [100]:
total_waste = solution_df["total_waste"].sum() + confirmatory_df["excess_length_produced"].sum()
total_length_needed = np.sum(df["length"] * df["quantity"])

In [101]:
total_waste

402.0

In [102]:
solution_df["number"].sum()

57.0

In [103]:
efficiency = 1 - total_waste / total_length_needed

In [104]:
print(f"We are {efficiency *100:.2f}% efficient")

We are 96.97% efficient


In [105]:
# Let's model this using a simple heuristic

In [106]:
solved_fn = fn.replace(".csv", "-solved.csv")

In [107]:
solution_df.to_csv(solved_fn)
print(f"wrote to {solved_fn}")

wrote to ./#4 Consolidated-solved.csv
